In [64]:
import pandas as pd

# read dataset
df=pd.read_csv("LoanStats3a.csv",skiprows=1)

#remove columns with more than 50% null values
rows=df.shape[0]
drop_na_columns=df.isna().sum()[df.isna().sum()>0.5*rows].index
df=df.drop(drop_na_columns,axis=1)
df=df.drop(['desc'],axis=1)

C:\Users\milin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
df=df.drop(['funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d','zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'],axis=1)

In [66]:
df=df[(df['loan_status']=='Charged Off' )|(df['loan_status']=='Fully Paid')]
loan_status_replace={"loan_status":{"Charged Off":0,"Fully Paid":1}}
df=df.replace(loan_status_replace)
df["loan_status"].value_counts()

1    34116
0     5670
Name: loan_status, dtype: int64

In [67]:
#drop columns with only one values
for col in df.columns:
    if len(df[col].dropna().unique())==1:
        df=df.drop(col,axis=1)

In [68]:
#drop rows with more than one percent missing values
df=df.drop(df.isna().sum()[df.isna().sum()>0.01*rows].index,axis=1)

In [69]:
#drop rows with missing values 
df=df.dropna()
df.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In [70]:
str_df=df.select_dtypes(include=['object'])

#some object dtype columns can be converted to float columns
def convertToFloat(v):
    v=v.replace("%","")
    v=v.strip()
    return float(v)

df[str_df.columns[8]]=str_df[str_df.columns[8]].apply(convertToFloat)

df[str_df.columns[1]]=str_df[str_df.columns[1]].apply(convertToFloat)



#as title and addr_stat columns contain more discrete values, drop them
df=df.drop(["title","addr_state","earliest_cr_line","last_credit_pull_d"],axis=1)


#make use of categorical columns by using dummy encoding
cat_col=["home_ownership", "verification_status", "purpose","term","debt_settlement_flag"]
dummies=pd.get_dummies(df[cat_col])
df=pd.concat([df,dummies],axis=1)
df=df.drop(cat_col,axis=1)
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'loan_status',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months', 'debt_settlement_flag_N', 'debt_settlement_flag_Y'],
      dtype='object')

In [71]:
#Let us use linear regression first and check if fpr is high then we can shift to a different model
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict,KFold
target="loan_status"
features=list(df.columns)
features.remove(target)

lr=LogisticRegression()
kf=KFold(df[features].shape[0],random_state=1)
predictions=cross_val_predict(lr,df[features],df[target],cv=kf)
tp=len(df[(predictions==1 )& (df[target]== 1)])
fp=len(df[(predictions==1 )& (df[target]== 0)])
tn=len(df[(predictions==0 )& (df[target]== 0)])
fn =len(df[(predictions==0 )& (df[target]== 1)])
print(tp/(fn+tp)," ",fp/(fp+tn))


0.999236910072787   0.9985843213590515


In [72]:
#fpr is high, and so apply RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
lr=RandomForestClassifier()
kf=KFold(df[features].shape[0],random_state=1)
predictions=cross_val_predict(lr,df[features],df[target],cv=kf)
tp=len(df[(predictions==1 )& (df[target]== 1)])
fp=len(df[(predictions==1 )& (df[target]== 0)])
tn=len(df[(predictions==0 )& (df[target]== 0)])
fn =len(df[(predictions==0 )& (df[target]== 1)])
print(tp/(fn+tp)," ",fp/(fp+tn))

0.9660131486264382   0.8917005839674393
